In [1]:
%pylab inline
import pandas as pd
import sklearn as skl
import scipy
import math
import sys
from collections import defaultdict
document = pd.read_excel("../News_SGU_31077_Processed_1.xlsx")
texts = document["News_Tokens"].apply(lambda x: x.split(" "))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
print(texts)

0        [май, состояться, общенародный, акция, бессмер...
1        [председатель, государственный, дума, федераль...
2        [заместитель, председатель, совет, безопасност...
3        [уважаемый, ветеран, университет, коллега, сту...
4        [саратовский, национальный, исследовательский,...
                               ...                        
31072    [международный, база, данные, обнародовать, ст...
31073    [результат, научный, исследование, заведовать,...
31074    [творческий, редакция, студенческий, издание, ...
31075    [председатель, правительство, российский, феде...
31076    [декабрь, год, диссертационный, совет, состоят...
Name: News_Tokens, Length: 31077, dtype: object


In [5]:
co_occurence = defaultdict(int)
word_cnt = defaultdict(int)
window_size = 6
for text in texts:
    for i, word in enumerate(text):
        word_cnt[word] += 1
        for j in range(max(0, i - window_size), min(len(text), i + window_size + 1)):
            if i != j:
                co_occurence[(word, text[j])] += 1

In [6]:
print(len(co_occurence))

13625675


In [7]:
total_cnt = len(word_cnt)
thesaurus = defaultdict(list)
for (word1, word2), cnt in co_occurence.items():
    p_w1w2 = cnt / total_cnt
    p_w1 = word_cnt[word1] / total_cnt
    p_w2 = word_cnt[word2] / total_cnt
    pmi = math.log2(p_w1w2 / (p_w1 * p_w2)) if p_w1w2 > 0 else -1
    npmi = pmi / -math.log2(p_w1w2)
    thesaurus[word1].append((word2, npmi))

In [8]:
print(len(thesaurus))

71192


In [10]:
threshold = 0.8
cnt = 0
result = []
for word in thesaurus:
    cnt += 1
    # print(thesaurus[word])
    filter = [(w, pmi) for w, pmi in thesaurus[word] if pmi > threshold]
    filter.sort(key=lambda x: -x[1])
    if len(filter) > 0:
        result.append([word, filter])
    # print(result[word])
    # if cnt > 5: break

df_results = pd.DataFrame(result, columns=["Word", "Most_Similar_Word"])

df_results.to_csv("similar_words_pmi.csv", index=False)